In [1]:
%autosave 60

Autosaving every 60 seconds


In [3]:
# Установка зависимостей (если нужно)
# Установка необходимых пакетов
# !pip install langchain langchain-community langchain-core langchain-ollama 
# !pip installpypdf faiss-cpu sentence-transformers ollama beautifulsoup4 
# !pip installunstructured lxml nltk tqdm dill gradio requests

In [5]:
#  На мак faiss-cpu через анаконду :
# conda install -c conda-forge faiss-cpu
# import faiss
# print(faiss.__version__)

Общий план:
1. Эмбеддинги — используем bge-m3 (только для векторизации!)
from langchain_ollama import OllamaEmbeddings
embeddings = OllamaEmbeddings(model="bge-m3")

2. LLM для генерации — используем модель, которая умеет отвечать
from langchain_ollama import ChatOllama
llm = ChatOllama(model="llama3", temperature=0.3)  # или mistral, phi3 и т.д.

3. Полученные чанки и вектора сохранены, можно загрузить сохранённые чанки и векторное хранилище, чтобы не перезапускать всю предобработку и векторизацию заново, достаточно раскомментировать и выполнить ячейку для перезапуска.

Материалы:
https://python.langchain.com/docs/tutorials/rag/

Курс по RAG
- https://www.youtube.com/playlist?list=PLfaIDFEXuae2LXbO1_PKyVJiQ23ZztA0x
- https://github.com/langchain-ai/rag-from-scratch

In [9]:
#TODOO Убедиться, что ollama запущен: В терминале: `ollama run llama3` или `ollama run bge-m3`

In [154]:
import os
import re
import ssl
import sys 
import time
import dill
import pickle
import requests
import zipfile
from typing import List
from bs4 import BeautifulSoup
from tqdm import tqdm
from urllib.parse import urljoin

# LangChain Core
from langchain_core.documents import Document
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# LangChain Text Splitting
from langchain.text_splitter import RecursiveCharacterTextSplitter

# LangChain Ollama Integration
from langchain_ollama import OllamaEmbeddings, ChatOllama

# LangChain Vector Store
from langchain_community.vectorstores import FAISS

# Gradio (UI)
import gradio as gr

In [ ]:
# # Ячейка для перезапуска
# # Убедитесь, что ollama запущена — без нее не загрузится OllamaEmbeddings
# # В терминале: ollama run bge-m3

# # Загрузка чанков
# if os.path.exists("output/scikit_chunks.pkl"):
#     with open("output/scikit_chunks.pkl", "rb") as f:
#         processed_chunks = dill.load(f)
#     print(f"✅ Загружено {len(processed_chunks)} чанков")
# else:
#     print("Чанки не найдены. Запустите предобработку.")

# # Загрузка векторного хранилища
# embeddings = OllamaEmbeddings(model="bge-m3")
# index_path = "output/scikit_faiss_index"

# if os.path.exists(index_path):
#     db = FAISS.load_local(index_path, embeddings, allow_dangerous_deserialization=True)
#     retriever = db.as_retriever(search_kwargs={"k": 4})
#     print("✅ Векторное хранилище загружено")
# else:
#     print(f"Индекс не найден: {index_path}")

# # Проверка
# if 'db' in locals():
#     docs = db.similarity_search("clustering", k=1)
#     print(f"\nПример извлечённого чанка:\n{docs[0].page_content[:200]}...")

# 1. Загрузка (Load)
Документация Scikit-learn https://scikit-learn.org/dev/versions.html 
- Актуальная версия называется stable-документация
- Наданный момент актуальна Scikit-learn 1.7.1

К сожалению, сайт scikit-learn.org блокирует попытки парсинга. Но зачем парсить, если они выкладывают актуальный zip документации? )

In [13]:
# Загружаем страницу со списком версий
versions_url = "https://scikit-learn.org/dev/versions.html"
response = requests.get(versions_url)
response.raise_for_status()
soup = BeautifulSoup(response.text, 'html.parser')

# Находим блок с "stable" и ищем ссылку на ZIP
zip_link = None
for p_tag in soup.find_all('p'):
    # Проверяем, есть ли упоминание "stable" в тексте
    if 'stable' in p_tag.get_text().lower() and 'documentation' in p_tag.get_text().lower():
        # Ищем ссылку на ZIP в том же теге <p>
        zip_anchor = p_tag.find('a', href=True, string=lambda text: text and 'zip' in text.lower())
        if zip_anchor:
            zip_link = urljoin(versions_url, zip_anchor['href'])
            break

if not zip_link:
    raise ValueError("❌ Не удалось найти ссылку на ZIP-архив stable-документации")

print(f"✅ Найдена ссылка на ZIP-архив: {zip_link}")

✅ Найдена ссылка на ZIP-архив: https://scikit-learn.org/stable//_downloads/scikit-learn-docs.zip


In [15]:
# Скачиваем ZIP
zip_path = "scikit-learn-docs.zip"
print(f"Скачиваю архив...")
with requests.get(zip_link, stream=True) as r:
    r.raise_for_status()
    with open(zip_path, 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)
print(f"Архив сохранён как {zip_path}")

# Распаковываем
import zipfile
docs_dir = "scikit-learn-docs"
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(docs_dir)
print(f"✅ Документация распакована в '{docs_dir}'")

Скачиваю архив...
Архив сохранён как scikit-learn-docs.zip
✅ Документация распакована в 'scikit-learn-docs'


# 2. Предобработка и разделение (Split)
https://python.langchain.com/docs/tutorials/rag/

1. Фильтрация файлов. Оставляет только полезные разделы, чтобы сократить итоговое кол-во чангов.  

2. Очистка контента
   - Удаляет навигацию, скрипты, стили через BeautifulSoup.  
   - Оставляет только основной текст (заголовки, описания, код).  

3. Извлечение метаданных 
   - Заголовки страниц, «хлебные крошки» (навигация вида User Guide → Model Selection).  
   - Приоритезация разделов (например, User Guide важнее API).  

4. Разбиение на чанки  
   - Использует RecursiveCharacterTextSplitter для семантического разделения текста.  
   - Сохраняет контекст (раздел, заголовок) в каждом чанке.  


In [17]:
class ScikitDocsPreprocessor:
    def __init__(self, docs_dir: str):
        self.docs_dir = docs_dir
        # Фильтрация ненужных разделов
        self.exclude_sections = ['search', 'genindex', '_sources', '_modules', 'changes']
        # Приоритет разделов для будущей сортировки/взвешивания
        self.section_hierarchy = {
            'user_guide': 1,
            'modules': 2,
            'auto_examples': 3,
            'api': 4
        }

    def extract_breadcrumbs(self, soup: BeautifulSoup) -> str:
        """Извлекает хлебные крошки для контекста навигации"""
        breadcrumbs = []
        for item in soup.select('.bd-breadcrumbs .nav-link'):
            text = item.get_text(strip=True)
            if text:
                breadcrumbs.append(text)
        return " → ".join(breadcrumbs) if breadcrumbs else "User Guide"

    def extract_page_title(self, soup: BeautifulSoup) -> str:
        """Извлекает заголовок страницы с приоритетом: og:title → .title → <title>"""
        og_title = soup.find('meta', property='og:title')
        if og_title:
            return og_title.get('content', '').strip()

        p_title = soup.find('p', class_='title')
        if p_title:
            return p_title.get_text(strip=True)

        if soup.title and soup.title.string:
            return soup.title.string.strip()

        return os.path.basename(self.docs_dir)

    def clean_html_content(self, soup: BeautifulSoup) -> str:
        """Очищает HTML от навигации, скриптов и оставляет только основной контент"""
        selectors_to_remove = [
            'nav', 'footer', 'script', 'style', 'aside', 'header', 'iframe', 'svg',
            'button', '.sidebar', '.toc', '.admonition', '.bd-header-article',
            '.bd-footer', '.bd-toc', '.pst-scrollable-table-container'
        ]
        for selector in selectors_to_remove:
            for el in soup.select(selector):
                el.decompose()

        # Поиск основного контента
        main = (
            soup.find('main') or
            soup.find('article') or
            soup.find('div', class_='bd-article-container') or
            soup.find('div', class_='document') or
            soup.body
        )

        if main:
            # Удаляем атрибуты для уменьшения шума
            for tag in main.find_all(True):
                tag.attrs = {}
            text = main.get_text()
        else:
            text = soup.get_text()

        # Нормализация пробелов
        text = re.sub(r'\s+', ' ', text).strip()
        return text

    def is_relevant_document(self, file_path: str) -> bool:
        """Фильтрует нерелевантные HTML-файлы"""
        return (
            file_path.endswith('.html') and
            not any(excl in file_path for excl in self.exclude_sections) and
            os.path.getsize(file_path) > 512  # Минимальный размер
        )

    def load_and_preprocess(self) -> List[Document]:
        """Загружает и предобрабатывает все HTML-файлы"""
        documents = []

        # Сбор всех HTML-файлов
        html_files = []
        for root, _, files in os.walk(self.docs_dir):
            for file in files:
                file_path = os.path.join(root, file)
                if self.is_relevant_document(file_path):
                    html_files.append(file_path)

        print(f"Найдено {len(html_files)} релевантных HTML-файлов")

        for file_path in tqdm(html_files, desc="Обработка HTML"):
            try:
                with open(file_path, 'r', encoding='utf-8') as f:
                    soup = BeautifulSoup(f, 'html.parser')

                # Извлечение контекста
                breadcrumbs = self.extract_breadcrumbs(soup)
                page_title = self.extract_page_title(soup)
                relative_path = os.path.relpath(file_path, self.docs_dir)
                section = relative_path.split(os.sep)[0] if os.sep in relative_path else 'root'

                # Очистка контента
                content = self.clean_html_content(soup)

                # Добавление семантического контекста
                context = (
                    f"[Breadcrumb: {breadcrumbs}]\n"
                    f"[Page: {page_title}]\n"
                    f"[Section: {section}]\n\n"
                )
                full_content = context + content

                metadata = {
                    'source': file_path,
                    'section': section,
                    'section_priority': self.section_hierarchy.get(section, 99),
                    'breadcrumbs': breadcrumbs,
                    'page_title': page_title,
                    'relative_path': relative_path
                }

                documents.append(Document(page_content=full_content, metadata=metadata))

            except Exception as e:
                print(f"\nОшибка при обработке {file_path}: {str(e)}")

        return documents

    def chunk_documents(self, documents: List[Document]) -> List[Document]:
        """Разбивает документы на чанки с сохранением семантики"""
        # Важно: используем правильные разделители для документации
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200,
            # Порядок важен: сначала по абзацам, потом по предложениям, потом по строкам
            separators=[
                "\n\n",           # Абзацы
                "\n",             # Параграфы
                ". ",             # Конец предложения
                "! ", "? ",
                " ",              # Слова
                ""                # По символам (в крайнем случае)
            ],
            keep_separator=True,
            strip_whitespace=True
        )

        print("Разбиение на чанки...")
        chunks = text_splitter.split_documents(documents)

        # Постобработка: очистка от артефактов
        for chunk in chunks:
            content = chunk.page_content

            # Удаление примечаний, ссылок и лишних пробелов
            content = re.sub(r'Note\s+.*?(?=\n\n|\Z)', '', content, flags=re.DOTALL)
            content = re.sub(r'\[\d+\]', '', content)  # Ссылки вроде [1]
            content = re.sub(r'\s+', ' ', content)     # Лишние пробелы
            content = re.sub(r'\n+', '\n', content).strip()

            chunk.page_content = content

        return chunks

    def process(self) -> List[Document]:
        """Полный пайплайн: загрузка → очистка → разбиение"""
        print("Запуск предобработки документации scikit-learn...")
        docs = self.load_and_preprocess()
        chunks = self.chunk_documents(docs)
        print(f"✅ Обработка завершена. Получено {len(chunks)} чанков.")
        return chunks

In [19]:
# Запуск
processor = ScikitDocsPreprocessor(docs_dir="scikit-learn-docs")
processed_chunks = processor.process()

Запуск предобработки документации scikit-learn...
Найдено 1080 релевантных HTML-файлов


Обработка HTML: 100%|█████████████████████████| 1080/1080 [02:26<00:00,  7.38it/s]


Разбиение на чанки...
✅ Обработка завершена. Получено 10602 чанков.


In [20]:
# Пример
for i, chunk in enumerate(processed_chunks[:2]):
    print(f"\n--- Пример {i+1} ---")
    print(f"Источник: {chunk.metadata['source']}")
    print(f"Контекст: {chunk.page_content[:300]}...")


--- Пример 1 ---
Источник: scikit-learn-docs/related_projects.html
Контекст: [Breadcrumb: User Guide] [Page: Related Projects] [Section: root]...

--- Пример 2 ---
Источник: scikit-learn-docs/related_projects.html
Контекст: Related Projects# Projects implementing the scikit-learn estimator API are encouraged to use the scikit-learn-contrib template which facilitates best practices for testing and documenting estimators. The scikit-learn-contrib GitHub organization also accepts high-quality contributions of repositories...


На данном этапе источник — это имя скаченного файла документации, в итоговом представлении преобразуем его в URL.

In [22]:
# Сохранение
import dill
os.makedirs("output", exist_ok=True)
with open("output/scikit_chunks.pkl", "wb") as f:
    dill.dump(processed_chunks, f)
print("✅ Чанки сохранены")

✅ Чанки сохранены


# 3. Векторизация (Embed) и FAISS

Чтобы сгенерировать эмбеддинги необходимо обратиться к локальному серверу Ollama, который должен быть запущен отдельно. Выполнить в терминале:

    ollama pull bge-m3
    ollama run bge-m3


In [33]:
# TODOO Так же можно протестировать и другие модели, например all-minilm, nomic-embed-text, mxbai-embed-large, но bge-m3 — одна из лучших для RAG. 

In [44]:
# Запустить в терминале: ollama run bge-m3

# Эмбеддинги
embeddings = OllamaEmbeddings(model="bge-m3")

# Построение FAISS
db = None
batch_size = 50
for i in tqdm(range(0, len(processed_chunks), batch_size), desc="Создание FAISS"):
    batch = processed_chunks[i:i + batch_size]
    if db is None:
        db = FAISS.from_documents(batch, embeddings)
    else:
        db.add_documents(batch, embeddings=embeddings)

print("✅ Векторное хранилище построено")

Создание FAISS: 100%|███████████████████████████| 213/213 [58:47<00:00, 16.56s/it]

✅ Векторное хранилище построено


In [73]:
# Сохранение векторов
def safe_save_faiss(db, folder_path):
    """Сохранение FAISS с обработкой проблемных метаданных"""
    for doc_id in db.index_to_docstore_id.values():
        doc = db.docstore._dict[doc_id]
        try:
            pickle.dumps(doc)
        except Exception:
            # Конвертируем все значения метаданных в строки (чтобы избежать ошибок сериализации)
            doc.metadata = {k: str(v) for k, v in doc.metadata.items()}
    
    # Увеличиваем лимит рекурсии для больших индексов иначе падает
    sys.setrecursionlimit(10000)
    
    # Сохраняем векторное хранилище
    db.save_local(folder_path)
    print(f"✅ Сохранено в {folder_path}")

# Вызов функции
safe_save_faiss(db, "output/scikit_faiss_index")

✅ Сохранено в output/scikit_faiss_index


# 4. Настройка Retriever и LLM

In [81]:
# Настройка Retriever
retriever = db.as_retriever(search_kwargs={"k": 4})  # 4 релевантных чанка

In [83]:
# Подключение LLM (через Ollama) 
# В терминале: ollama run llama3 - держать запущенным во время работы RAG-интерфейса.
from langchain_ollama import ChatOllama

llm = ChatOllama(model="llama3", temperature=0.3)

# 5. Сборка RAG-цепочки (Chain)

Для улучшения тестируемых промтов использован https://neuralwriter.com/ru/prompt-tool/

In [119]:
template = """Вы — эксперт по машинному обучению и библиотеке scikit-learn.
Отвечайте на вопрос на русском языке, опираясь только на следующий контекст.
Если в контексте нет ответа, скажите: "На основе документации scikit-learn ответить не могу."

Контекст:
{context}

Вопрос:
{question}

Ответ:"""

prompt = PromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [155]:
# Функция для преобразования локального пути в URL
def local_path_to_url(local_path: str, base_url: str = "https://scikit-learn.org/stable") -> str:
    relative_path = "/".join(local_path.split("/")[1:])  # убираем 'scikit-learn-docs/'
    return f"{base_url.rstrip('/')}/{relative_path}"

In [156]:
# Тестируем:

# Пример
local_path = "scikit-learn-docs/modules/clustering.html"
url = local_path_to_url(local_path)
print(url)

https://scikit-learn.org/stable/modules/clustering.html


# 6. Демонстрация: 3 примера с анализом

 ## Пример 1: Поиск факта

In [171]:
question = "Какой параметр в KMeans отвечает за количество кластеров?"
response = rag_chain.invoke(question)

docs = db.similarity_search(question, k=2)
sources = "\n".join(f"- {local_path_to_url(d.metadata['source'])}" for d in docs)

print(f"Ответ: {response}\n\nИсточники:\n{sources}")

Ответ: Параметр `n_clusters` в классе `KMeans` отвечает за количество кластеров. Он имеет тип `int` и по умолчанию равен 8.

Источники:
- https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html
- https://scikit-learn.org/stable/auto_examples/text/plot_document_clustering.html


✅ Анализ:

Запрос прямой, факт содержится в докумментации. RAG система точно извлекла информацию из правильного источника. Это демонстрирует хорошую точность ретривера и качество эмбеддингов.

Первый источник — sklearn.cluster.KMeans.html — идеален: это прямая API-документация, где описан параметр n_clusters.
Второй источник — пример кластеризации текстов — релевантен, но косвенный: используется KMeans, но не объясняется параметр. Тем не менее, это хороший контекст.

 ## Пример 2: Синтез информации

In [174]:
question = "Чем отличаются KMeans и MiniBatchKMeans?"
response = rag_chain.invoke(question)

docs = db.similarity_search(question, k=2)
sources = "\n".join(f"- {local_path_to_url(d.metadata['source'])}" for d in docs)

print(f"Ответ: {response}\n\nИсточники:\n{sources}")

Ответ: КMeans и MiniBatchKMeans - это оба алгоритмы для кластеризации, но они отличаются по своей природе.

KMeans - это классический алгоритм кластеризации, который учитывает все данные в выборке. Он работает с помощью итеративного процесса, где центроиды кластеров обновляются на основе средних значений для каждого кластера. Каждый шаг алгоритма занимает время O(n), где n - количество данных.

MiniBatchKMeans - это модифицированная версия KMeans, которая работает с mini-батчами (мини-выборками) из данных, а не со всеми данными сразу. Это позволяет ускорить процесс кластеризации, особенно для больших выборок. MiniBatchKMeans также использует итеративный процесс, но каждый шаг занимает время O(m), где m - размер mini-батча.

В целом, KMeans обеспечивает более точные результаты, но может быть медленнее, чем MiniBatchKMeans, для больших выборок. MiniBatchKMeans, с другой стороны, может быть быстрее, но может привести к менее точным результатам из-за ограничения mini-батча.

Источники:
- h

✅  Анализ:


Ответ корректно отражает ключевые различия:   Точно описаны основные шаги KMeans, правильно отмечен компромисс между скоростью и точностью применении MiniBatchKMeans.  В целом хорошее сравнение. 


 ## Пример 3: Сложный/негативный случай

 Спрашиваем информацию, которой нет в документации.

In [175]:
question = "Как scikit-learn поддерживает обучение с подкреплением?"
response = rag_chain.invoke(question)

docs = db.similarity_search(question, k=2)
sources = "\n".join(f"- {local_path_to_url(d.metadata['source'])}" for d in docs)

print(f"Ответ: {response}\n\nИсточники:\n{sources}")

Ответ: На основе документации scikit-learn ответить не могу.

Источники:
- https://scikit-learn.org/stable/faq.html
- https://scikit-learn.org/stable/related_projects.html


✅ Анализ:

LLM не галлюцинирует, а честно говорит, что не знает — хорошее поведение.
Ответ верно отражает, что scikit-learn изначально не поддерживает обучение с подкреплением, приведена корректная цитата из FAQ. 
Указаны релевантные разделы документации

# Пример 3.2
Еще один сложный случай: домен программирования, объемный запрос, который не имеет единого ответа в документации — это композиция знаний из разных разделов.

Запрос: Создай полный пайплайн для работы с моделью классификации, включающий загрузку и предобработку данных с учетом дисбаланса классов, кросс-валидацию на 5 фолдах и получение предсказаний , используя все возможности scikit-learn. Прокомментируте свой выбор методов. 

In [181]:
question = "Создай полный пайплайн для работы с моделью классификации, включающий загрузку и предобработку данных с учетом дисбаланса классов, кросс-валидацию на 5 фолдах и получение предсказаний , используя все возможности scikit-learn. Прокомментируте свой выбор методов."
response = rag_chain.invoke(question)

docs = db.similarity_search(question, k=4)
sources = "\n".join(f"- {local_path_to_url(d.metadata['source'])}" for d in docs)

print(f"Ответ: {response}\n\nИсточники:\n{sources}")

Ответ: Вот полный пайплайн для работы с моделью классификации, включающий загрузку и предобработку данных с учетом дисбаланса классов, кросс-валидацию на 5 фолдах и получение предсказаний:

```
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, ImbalancedDatasetSampler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Загрузка данных
iris = load_iris()
X, y = iris.data, iris.target

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

# Создание пайплайна с использованием StandardScaler и LogisticRegression
pipe = Pipeline([
    ('standardscaler', StandardScaler()),
    ('logisticregression', LogisticRegression())
])

# Фитинг пайплайна на обучающей выборке
pipe.fit(X_train, y_train)

# Кросс-валидация на 5 фолдах
scores = cross_

✅ Анализ:


Вот детальный анализ ответа RAG-системы на запрос о пайплайне классификации:

Основные проблемы ответа: 

1. **Фактические ошибки**:
   - **ImbalancedDatasetSampler** не существует в scikit-learn (ошибка генерации)
 Набор данных Iris не имеет дисбаланса классов, но модель этого действительно не знает, взяла датасет, на котором в документации показана работа с классами. 

2. **Неполное решение**:

 - Нет стратификации при разделении на train и test
  - Нет реальных методов обработки дисбаланса (SMOTE, классовые веса и т.д.)
  - Отсутствует подбор гиперпараметров (хотя при других генерациях предлагалось использовать GridSearchCV)
  - Нет других метрик, кроме accuracy (максимально неэффективная метрика при дисбалансе :) )
    
4. **Проблемы с источниками**:
 Ссылки слишком общие (разделы getting_started и data_transforms), почему-то при данной генерации не рассматривался, например более конкретный  раздел https://scikit-learn.org/stable/modules/generated/sklearn.utils.class_weight.compute_class_weight.html   
 

# Gradio-интерфейс для презентации RAG-системы

In [79]:
# TODO В терминале: ollama run llama3 - держать запущенным во время работы RAG-интерфейса.

Добавим функцию преобразования локального пути в URL

In [161]:
def answer(question, progress=gr.Progress()):
    if not question.strip():
        return "Введите вопрос по scikit-learn."
        
    try:
        progress((1, 2), "Поиск в базе знаний...")
        time.sleep(0.1)  # минимальная задержка, чтобы прогресс отобразился

        # Поиск релевантных чанков
        docs = db.similarity_search(question, k=2)

        progress((2, 2), "Генерация ответа...")
        time.sleep(0.1)

        # Генерация ответа
        response = rag_chain.invoke(question)

        # Форматируем источники как кликабельные ссылки
        sources = "\n".join([
            f"🔗 [{os.path.basename(d.metadata['source'])}]({local_path_to_url(d.metadata['source'])})"
            for d in docs
        ])
        
        return f"{response}\n\n---\n**Источники:**\n{sources}"

    except Exception as e:
        return f"❌ Ошибка: {str(e)}"


In [153]:
demo = gr.Interface(
    fn=answer,
    inputs=gr.Textbox(
        placeholder="Например: Какой параметр в KMeans отвечает за количество кластеров?",
        label="Ваш вопрос",
        lines=2
    ),
    outputs=gr.Markdown(label="Ответ"),
    title="🔍 RAG по документации scikit-learn",
    description="""
    Задайте вопрос о кластеризации, preprocessing, моделях и других аспектах scikit-learn.
    Система найдёт ответ в официальной документации.
    """,
    examples=[
        ["Какой параметр в KMeans отвечает за количество кластеров?"],
        ["Чем отличаются DBSCAN и KMeans?"],
        ["Почему scikit-learn не поддерживает графовые нейронные сети?"]
    ],
    theme=gr.themes.Soft(),
    flagging_mode="never",
    # allow_flagging="never",
    clear_btn="Очистить",
    submit_btn="Отправить"
)

demo.launch(debug=True, share=False)

* Running on local URL:  http://127.0.0.1:7869
* To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.


Создайте полный пайплайн для работы с моделью классификации, включающий загрузку и предобработку данных с учетом дисбаланса классов, кросс-валидацию на 5 фолдах и получение предсказаний , используя все возможности scikit-learn.

-----